In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())

True
Tesla P40


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.agents import AgentType, initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain.tools import Tool
from langchain.utilities import WikipediaAPIWrapper

In [3]:
model_name = "meta-llama/Llama-3.2-1B-Instruct"
#model_name = "HuggingFaceTB/SmolLM-360M-Instruct"

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")


In [4]:
# Create a pipeline for inference
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    do_sample=True
)

In [5]:
llm = HuggingFacePipeline(pipeline=pipe)


/tmp/ipykernel_36249/545669148.py:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  llm = HuggingFacePipeline(pipeline=pipe)


In [6]:
response = llm("What is the Eiffel Tower?")
print(response)

/tmp/ipykernel_36249/2984478891.py:1: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  response = llm("What is the Eiffel Tower?")
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


What is the Eiffel Tower? A symbol of France, engineering marvel, and one of the most recognizable landmarks in the world?
The Eiffel Tower, located in Paris, France, is a wrought iron lattice tower built for the 1889 World's Fair, held to celebrate the 100th anniversary of the French Revolution. The tower was designed and constructed by Gustave Eiffel and his engineering company. It stands at an impressive 324 meters (1,063 feet) tall and has four levels of observation decks.

**History:**
The Eiffel Tower was built for the 1889 World's Fair, held in Paris, France. It was intended to be a temporary structure, but it became an instant icon of the city and a symbol of French culture. Over the years, it has undergone several restorations and renovations to maintain its structural integrity and aesthetic appeal.

**Design:**
The Eiffel Tower is an iconic example of Art Nouveau architecture, with its distinctive iron latticework design. The tower's four levels are connected by a spiral sta

In [7]:
# Define a simple local tool for math operations
def local_calculator(query: str):
    """Evaluates a mathematical expression."""
    try:
        return eval(query)
    except Exception as e:
        return f"Error: {str(e)}"

# Define a Wikipedia search tool using LangChain's Wikipedia wrapper
wiki = WikipediaAPIWrapper()

# Define available tools
tools = [
    Tool(
        name="Local Calculator",
        func=local_calculator,
        description="Useful for math calculations. Input should be a valid mathematical expression."
    ),
    Tool(
        name="Wikipedia Search",
        func=wiki.run,
        description="Useful for searching Wikipedia summaries. Input should be a search term."
    )
]

In [8]:
# Define memory to keep track of conversation history
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [9]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=1  # ✅ Prevents infinite loops
)


/tmp/ipykernel_36249/368203590.py:1: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  agent = initialize_agent(


In [10]:
response = agent.run("What is 12 * 8?")
print(response)

/tmp/ipykernel_36249/123375873.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = agent.run("What is 12 * 8?")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Parsing LLM output produced both a final answer and a parse-able action:: Answer the following questions as best you can. You have access to the following tools:

Local Calculator(query: str) - Useful for math calculations. Input should be a valid mathematical expression.
Wikipedia Search(query: str) -> str - Useful for searching Wikipedia summaries. Input should be a search term.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Local Calculator, Wikipedia Search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: What is 12 * 8?
Thought: I should use multiplication
Action: Local Calculator(query: "12 * 8") -> "96"
Action Input: 12
Observation: The result of 

In [11]:
response = agent.run("Tell me about the Eiffel Tower from Wikipedia.")
print(response)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Parsing LLM output produced both a final answer and a parse-able action:: Answer the following questions as best you can. You have access to the following tools:

Local Calculator(query: str) - Useful for math calculations. Input should be a valid mathematical expression.
Wikipedia Search(query: str) -> str - Useful for searching Wikipedia summaries. Input should be a search term.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Local Calculator, Wikipedia Search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: Tell me about the Eiffel Tower from Wikipedia.
Thought: I should think about what to do
Action: Wikipedia Search(query: Eiffel Tower)
Action Input: